In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Installs

In [12]:
!pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0
!pip install datasets==2.16.1 PyPDF2==3.0.1 langchain_experimental python-dotenv==1.0.1 pyyaml==6.0.1 coloredlogs==15.0.1 mdc==1.2.1 pytest==8.1.2
!pip install -r drive/MyDrive/RAG-llamaindex/requirements.txt
!pip install huggingface_hub
!pip install llama-index-llms-huggingface
!pip uninstall -y bitsandbytes accelerate transformers
!pip install bitsandbytes accelerate transformers


Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: accelerate 0.29.3
Uninstalling accelerate-0.29.3:
  Successfully uninstalled accelerate-0.29.3
Found existing installation: transformers 4.40.2
Uninstalling transformers-4.40.2:
  Successfully uninstalled transformers-4.40.2
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
  Using cached transformers-4.42.3-py3-none-any.whl (9.3 MB)


#Imports


In [25]:
# For Loading the LLM
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

import warnings
warnings.filterwarnings('ignore')

# For Data Ingestion
from typing import Literal, Any, List
import logging
import PyPDF2
import sys
sys.path.append('drive/MyDrive/RAG-llamaindex')
import os
from langchain_experimental.text_splitter import SemanticChunker
from math import ceil

# For Embeddings
from transformers import AutoModel

# For the LLM Model
from llama_index.llms.huggingface import HuggingFaceLLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


#Loading the LLM

In [ ]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

import warnings
warnings.filterwarnings('ignore')

In [20]:
# Getting the Hugging Face token
config_data = json.load(open("drive/MyDrive/RAG-llamaindex/config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

# Defining the pre-trained model we will use, which is Llama-3-8B
model_name = "meta-llama/Meta-Llama-3-8B"

# Defining the quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Defing the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name ,
                                          token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

# Initialising the LLM
model = AutoModelForCausalLM.from_pretrained(
    model_name ,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

#Data Ingestion

In [23]:
from typing import Literal, Any, List
import logging
import PyPDF2
import sys
sys.path.append('drive/MyDrive/RAG-llamaindex')
import os
from langchain_experimental.text_splitter import SemanticChunker
from math import ceil

In [24]:
reader = SimpleDirectoryReader(input_dir="/content/drive/MyDrive/RAG-llamaindex/Data")
documents = reader.load_data(num_workers=4)

# Filter out the data where the file_name is not "Harry Potter The Complete Collection.pdf"
documents = [doc for doc in documents if doc.metadata["file_name"] == "Harry Potter The Complete Collection.pdf"]

# Creating a function for chunking using the LLama Index SentenceSplitter
def get_chunks(
    documents: list[Any],
    chunk_size: int = 512,
    chunk_overlap: float = 0.0
) -> list[str]:
    overlap = int(chunk_size * chunk_overlap)
    text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks= text_splitter.get_nodes_from_documents(documents, show_progress=True)

    return chunks

# Creating a function for manual chunking
def get_chunks_manual(documents: list[Any], chunk_size: int = 512, chunk_overlap: float = 0.0) -> List[str]:
    """
    Takes in a `file_path`, retrieves the document, breaks it down into chunks of size
    `chunk_size` and overlap `chunk_overlap`, and returns the chunks.
    """
    chunks = []

    text = ""
    for doc in documents:
        text += doc.text
    # replace every \t with " "
    text = text.replace("\t", " ")

    step_size = ceil(chunk_size * (1 - chunk_overlap))
    for i in range(0, len(text), step_size):
        chunks.append(text[i:i + chunk_size])

    return chunks

KeyboardInterrupt: 

In [ ]:
# Using the Sentence Splitter

# Creating chunks of 512 characters with no overlap
chunks_512_0 = get_chunks(documents)
print("Created", len(chunks_512_0), "chunks of size 512 and 0% overlap with the SentenceSplitter")
print("The 100th chunk is:", chunks_512_0[100])

# Creating chunks of 512 with 50% overlap
chunks_512_50 = get_chunks(documents, chunk_size=512, chunk_overlap=0.5)
print("Created", len(chunks_512_50), "chunks of size 512 and 0% overlap with the SentenceSplitter")
print("The 100th chunk is:", chunks_512_50[100])

In [ ]:
# Using the Manual Chunking

# Creating chunks of 512 with no overlap
manual_chunks_512_0 = get_chunks_manual(documents, 512, 0.0)
print("Created", len(manual_chunks_512_0), "chunks of size 512 and 0% overlap with the Manual Splitter")
print("The 100th chunk is:", manual_chunks_512_0[100])

# Creating chunks of 512 with 50% overlap
manual_chunks_512_50 = get_chunks_manual(documents, 512, 0.5)
print("Created", len(manual_chunks_512_50), "chunks of size 512 and 0% overlap with the Manual Splitter")
print("The 100th chunk is:", manual_chunks_512_50[100])

#Creating the Embedding Model

In [26]:
from transformers import AutoModel

In [27]:
class HuggingFaceEmbedding:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def get_text_embedding(self, text: str) -> torch.Tensor:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze()

    def get_embeddings(self, texts: list[str], batch_size: int = 16) -> list[torch.Tensor]:
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            inputs = self.tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings.extend(batch_embeddings.cpu())
        return embeddings

embed_model =  HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

#Defining the LLM Model

In [32]:
from llama_index.llms.huggingface import HuggingFaceLLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
llm= HuggingFaceLLM(model=model, tokenizer=tokenizer)

# Find the embedding of each text in Manual_Chunks of size 512 and 0% overlap and put in list
manual_512_0_embed = embed_model.get_embeddings(manual_chunks_512_0)
manual_512_0_embed = [embedding.numpy() for embedding in manual_512_0_embed]
print(len(manual_512_0_embed))

# Find the embedding of each text in Manual_Chunks of size 512 and 50% overlap and put in list
manual_512_50_embed = embed_model.get_embeddings(manual_chunks_512_50)
manual_512_50_embed_np = [embedding.numpy() for embedding in manual_512_50_embed]
print(len(manual_512_50_embed_np))

In [ ]:
# Creating the first Retriever: Top-k Embeddings Retriever

def top_k_embed_retriever(query: str, k: int, embed_model, manual_embed_np, texts):
    query_embed = embed_model.get_text_embedding(query)
    query_embed_np = query_embed.cpu().numpy().reshape(1, -1)

    # Calculate cosine similarity with all the other embeddings
    similarities = cosine_similarity(query_embed_np, manual_embed_np).flatten()

    # Find the top k indices
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]

    # Retrieve the top k texts
    top_k_texts = [texts[i] for i in top_k_indices]

    return top_k_texts

# Example usage
query = "Hogwarts"
k = 3
top_k_texts = top_k_embed_retriever(query, k, embed_model, np.stack(manual_embed_np), manual_chunks)
print(top_k_texts[0])
print("___________")
print(top_k_texts[1])
print("___________")
print(top_k_texts[2])

In [ ]:
vectorizer_512_0 = TfidfVectorizer()
vectorizer_512_50 = TfidfVectorizer()

tfidf_matrix_512_0 = vectorizer_512_0.fit_transform(manual_chunks)
tfidf_matrix_512_50 = vectorizer_512_50.fit_transform(manual_chunks_512_50)

In [ ]:
# Creating the second Retriever: Top-k TF-IDF Retriever
def top_k_tfidf_retriever(query: str, k: int, tfidf_matrix, texts, vectorizer):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_k_indices = np.argpartition(similarities, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]
    top_k_texts = [texts[i] for i in top_k_indices]
    return top_k_texts

# Example usage
query = "Hogwarts"
k = 3
top_k_texts = top_k_tfidf_retriever(query, k, tfidf_matrix_512_0, manual_chunks, vectorizer_512_0)
print(top_k_texts[0])
print("___________")
print(top_k_texts[1])
print("___________")
print(top_k_texts[2])